In [8]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
#from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import BatchNormalization
#from keras_preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import joblib
#test23

In [9]:
#train_dir="../input/face_recognition_dataset/Original_Images/Original_Images/"
#train_dir="/home/meatisdelicious/DetectionVisage/DetectionVisage/FaceDetectionV4/input/face_recognition_dataset/Original_Images/Original_Images/"
#Unmodified Dataset
train_dir="Original Images/Original Images"
#/home/meatisdelicious/DetectionVisage/DetectionVisage/FaceDetectionV4/face_recognition_dataset/Original_Images
#Modified Dataset
#train_dir="/face_recognition_dataset/Original_Images_more/"

# Quand on a une classification à faire.
generator = ImageDataGenerator()
#seed : melange tjr de la meme manière le dataset à partir du meme argument
dataset = tf.keras.utils.image_dataset_from_directory(train_dir,image_size=(224, 224),batch_size=32, shuffle=True, seed=123)
dataset_size = len(dataset)
train_test_split=0.8
train_ds=dataset.take(int(train_test_split*dataset_size))
test_ds=dataset.skip(int(train_test_split*dataset_size))

classes = dataset.class_names

Found 2562 files belonging to 31 classes.


In [10]:
model = Sequential()
model.add(Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(len(classes),activation='softmax'))
# 20 couches


In [11]:
model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = 'adam',
    metrics=['binary_accuracy', 'categorical_accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 batch_normalization_5 (Batc  (None, 111, 111, 32)     128       
 hNormalization)                                                 
                                                                 
 conv2d_6 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                      

In [12]:
#history = model.fit(train_ds,epochs= 30, batch_size=32)
history = model.fit(train_ds,epochs= 15, batch_size=32, validation_data=test_ds)

Epoch 1/15
12/64 [====>.........................] - ETA: 1:30 - loss: 3.9261 - binary_accuracy: 0.0182 - categorical_accuracy: 0.0260

KeyboardInterrupt: 

In [ ]:
# Saving Model
filename = 'model1_original_more_dataset.hdf5'
#joblib.dump(history, filename)

In [ ]:
# load the model from disk
model_saved = joblib.load(filename)
#model_saved.save('model_pres.h5')
joblib.dump(model_saved, 'model_pres.h5')
#tf.keras.models.save_model(model_saved,"model_pres.h5")

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-01-01 23:18:36         8355
metadata.json                                  2023-01-01 23:18:36           64
variables.h5                                   2023-01-01 23:18:36      3059096
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers
......batch_normalization
.........vars
............0
............1
............2
............3
......batch_normalization_1
.........vars
............0
............1
............2
............3
......batch_normalization_2
.........vars
............0
............1
............2
............3
......batch_normalization_3
.........vars
............0
............1
............2
............3
......batch_normalization_4
.........vars
............0
............1
............2
............3
......conv2d
.........vars
............0
............1
......conv2d_1
.........vars
...

['model_pres.h5']

In [ ]:
plt.plot(model_saved.history['categorical_accuracy'])
plt.plot(model_saved.history['loss'])
plt.xlabel('Time')
plt.legend(['categorical_accuracy', 'loss'])
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

def predict_image(image_path):
    img = image.load_img(image_path, target_size=(224,224,3))
    plt.imshow(img)
    plt.show()
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    
    pred = model.predict(images, batch_size=32)
    
    # Prediction accuracy
    #--> le pb que j'arrive pas à resoudre est ici
    print("Score:",(np.argmax(pred)),"%")
    model.evaluate(test_ds)

    print("Actual result: "+(image_path.split("/")[-1]).split("_")[0])
    print("Predicted result: "+classes[np.argmax(pred)])

predict_image("/home/meatisdelicious/DetectionVisage/DetectionVisage/FaceDetectionV4/input/face_recognition_dataset/Original_Images/Original_Images/Brad Pitt/Brad Pitt_103.jpg")

In [ ]:
predict_image("/home/meatisdelicious/DetectionVisage/DetectionVisage/FaceDetectionV4/input/face_recognition_dataset/Original_Images/Original_Images/Camila Cabello/Camila Cabello_47.jpg")